
<p style="text-align:center;font-size:50px"><b>Bertologia</b></p>


<img src="https://github.com/alan-barzilay/NLPortugues/blob/master/imagens/logo_nlportugues.png?raw=true"  style="height:45%" align="left">

<img src="http://ccsl.ime.usp.br/files/unmanaged/logos/logoIME+Texto.png"  style="height:45%;width:10%" align="right">

## Agenda

   - Momento Computer Vision em NLP: ULMFit
   - Open AI Transformer
   - BERT
   - GPT-1 vs GPT-2 vs GPT-3 
   - Bertologia
   
   (a ordem deveria ser essa?)

# Momento Computer Vision em NLP

## Transfer Learning

Um dos grandes motivos responsáveis para o _boom_ de Deep Learning em meados de 2014 foi uma técnica conhecida como **Transfer Learning**. Ela pode ser definida como a seguinte:

> Transfer learning and domain adaptation refer to the situation where what has been learned in one setting … is exploited to improve generalization in another setting

[Deep Learning Book](https://www.deeplearningbook.org/contents/representation.html)


<center><img src='images/transfer_learning.jpeg'></center>

Então, basicamente, a gente treina uma rede neural pesada para uma task grande, como o ImageNet, **pegamos os pesos da rede treinada** e treinamos uma nova rede para uma task menor, inicializando ela com esses pesos aprendidos na tarefa anterior

## Mas como isso é possível em NLP?

A ideia de combinar a ideia de combinar transfer learning com o que ficou conhecido como aprendizado auto-surpervisionado.

> Self-supervised learning: Training a model using labels that are embedded in the independent variable, rather than requiring external labels. For instance, training a model to predict the next word in a text.

[Fast.ai book- Chapter 10](https://github.com/fastai/fastbook/blob/master/10_nlp.ipynb)

A tarefa, no caso de NLP, um tipo de modelo que entra nessa categoria de aprendizado auto surpervisionado é conhecida como **modelo de linguagem**

### Modelo de Linguagem

Um modelo de linguagem é capaz de determinar a _fluência_ de um texto. Na prática, o desafio é, **com base nas palavras que eu vi até agora, qual deve ser a próxima palavra?**

$$
\begin{array}{c}
P(w_n|w_0, w_1, w_2, \dots, w_{n-1})
\end{array}
$$

Qual a intuição para isso funcionar?

#### Regra da Cadeia

$$
\begin{array}{c}
P(w_0, w_1, w_2, \dots, w_n) = P(w_0) \times P(w_1|w_0) \times P(w_2|w_0, w_1)  \dots \times P(w_n|w_0, w_1, w_2, \dots, w_{n-1})
\end{array}
$$

$$
\begin{array}{c}
P(w_0, w_1, w_2, \dots, w_n) = \displaystyle \prod_{i} P(w_n|w_0, w_1, w_2, \dots, w_{n-1})
\end{array}
$$

P("Eu gosto de abacate") = P("Eu") $\times$ P("gosto|Eu") $\times$ P("de|Eu, gosto") $\times$ P("abacate|Eu, gosto, de")

Percebam que tendo um modelo assim, a gente **não precisa de labels** e, logo, o próprio dado de treinamento pode ser usado para gerar as labels e, assim, treinar algo! Daí a ideia de **auto surpervisionado**

<img src='images/yann_lecunn_quote.jpg'  style="height:70%;width:40%" align="left">

<img src='images/yann_lecunn_slides.jpg' align="right">

## ULMFit

Em Janeiro de 2018, Sebastian Ruder e Jeremy Howard tiveram a ideia de combinar essas duas técnicas, modelo que eles nomearam de [**ULMFit**](https://arxiv.org/abs/1801.06146) ou _Universal Language Model Fine-tuning_

![](images/ulmfit.png)
[fastai book](https://github.com/fastai/fastbook/blob/master/10_nlp.ipynb)

A ideia é criar um modelo de Linguagem "genérico", treinado com base no Wikipedia. Disso, fazemos uma etapa de fine-tuning no nosso dataset objetivo, **com a ideia de que o modelo aprenda o linguajar específico desse domínio** e, então, fazemos uma etapa final de treinamento _focando na nossa tarefa final_, como um classificador de sentimentos de filmes por exemplo.

## Mas e a Arquitetura?

Para treinar esses modelos, foi utilizada a seguinte arquitetura:

- [AWD LSTM](https://arxiv.org/pdf/1708.02182.pdf), que pode ser entendida como uma LSTM com algumas tecnicas a mais de otimização e regularização :)

<center><img src='images/ulmfit_example.png'  style="height:70%;width:70%"></center>

[overview ulmfit](https://humboldt-wi.github.io/blog/research/information_systems_1819/group4_ulmfit/#overviewulmfit)

Por fim, a loss, utilizada é a boa e velha _Cross Entropy Loss_ :)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# define model
vocab_size = 1000
embed_dim = 50  # tamanho do Embedding de cada token
maxlen = 200  # Tamanho máximo da sentença

In [2]:
# Seu código aqui
tokenizer_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace', ngrams=None, output_mode="int",
    output_sequence_length=maxlen, pad_to_max_tokens=False)
tokenizer_layer.adapt(np.array(["O rato roeu a roupa do rei de roma"]))

inputs = layers.Input(shape=(1,), dtype=tf.string, name='input_text')
embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)

In [3]:
x = tokenizer_layer(inputs)    
x = embedding_layer(x)
x = layers.LSTM(units=1)(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [4]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

# Mas atenção não é tudo que a gente precisa?

Contudo, lembrem que na útlima aula vimos um paper que saiu em no meio de 2017, o **Attention is All you need**? Nele, foi apresentado a arquitetura Transformer



<center><img src='images/transformer.png'  style="height:20%;width:20%"></center>


Acontece que a ideia tida por [Radford et. al](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf) em Junho de 2018 foi a de usar a parte **decoder** da rede para o processo de aprendizado auto surpervisionado! Esse modelo ficou conhecido como **Open Ai Transformer**, ou GPT-1

<center><img src='images/transformer-decoder.png'  style="height:20%;width:20%"></center>

[attention? attention!](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#decoder)

* A diferença é que no caso desse modelo do Open AI $N=12$ :)

In [14]:
from IPython.display import Video

Video("images/TransformerDecoderExample.mp4")

Basicamente, então, também é valido dizer que o GPT é um modelo **auto regressivo**. Ou sejam a medida que ele aprende uma palavra, **ele usa ela como informação para a próxima**

<center><img src='http://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif'></center>

[Illustrated GPT-2](http://jalammar.github.io/)

As diferenças entre os GPT's são arquiteturais, como veremos daqui a pouco :)


## Fine Tuning

Seguindo, a ideia de pós treinamento auto surpervisionado, os autores propuseram fazer fine-tunings específicos, dependendo do tipo de task utilizada

**PERGUNTA para o Finger: qual é a entrada da rede finetuning? Eu entendi que era o token [CLS] no caso do BERT, mas aqui isso nao ficou claro pra mim se é um token só ou as somas dos embeddings**

<center><img src='images/open-ai-finetuing.png'></center>

[open AI transformer paper](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)

A diferença aqui, é que no processo de Finetuing, dois novos Embeddings são adicionados: o de _start_ **\<s\>** e _extract_ **\<e\>**

## E aí a grande revolução em NLP começou :)

<center><img src='images/open-ai-finetuning-results.png' style="height:50%;width:50%"></center>
    
[open AI transformer paper](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)

# BERT

Apesar do artigo do GPT-1 ser impressionante para a época, alguns meses depois, em Outubro do mesmo ano (o GPT foi lançado em Junho), o pessoal do Google propôs usar a parte **encoder** do transformer, junto de outras modificações leves, dando ao modelo o nome de **B**idirectional **E**ncoder **R**epresentations from **T**ransformers (BERT)

## Encoder Layer

Relembrando o Encoder Layer dos Transformers

<center><img src='images/transformer-encoder.png'  style="height:30%;width:30%"></center>

[attention? attention!](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#encoder)

* A diferença é que no caso desse modelo do BERT $N=12$ ou $N=24$ :)

A primeira diferença é **abandonar o Mask Multi Head** e usar direto o **Multi Head Attention**, daí o nome birecional!


Mas se fazemos isso, não estamos **overfittando** o modelo já que vamos olhar o contexto futuro para prever a palavra atual?

Solução: usar **outra loss**

Falta:
- Loss: Masked Language Model Loss e NSP loss
- GPT-2 vs GPT-3 (fala do Sparse Transformer(?))
- Roberta; Albert; ELECTRA; T5; BART, passando por cima de todos
- HuggingFace code examples
- Tricks for training in GPUs: fp16; grandient accumulation